In [ ]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

Rozwiązanie zagadnienia kinematyki odwrotnej, podobnie jak zagadnienia kinematyki prostej, trzeba rozpocząć od stworzenia modelu manipulatora.

In [ ]:
L1 = rtb.DHLink(d=1.0, alpha=pi/2, theta=0.0, a=0.5)
L2 = rtb.DHLink(theta=0.0, a=0.7)
robot = rtb.DHRobot([L1, L2])

W następnym kroku konieczne jest określenie położenia i orientacji końcówki manipulatora, dla której zagadnienie  ma zostać rozwiązane. Pozycję tą należny przedstawić w postaci macierzy homogenicznej. Jednym z najprostszych sposobów jest określenie macierzy translacji i rotacji. Macierz translacji tworzy się wykorzystując polecenie **SE3** jak w przykładzie:
```py
trans = SE3(0.1, 0.2, 0.3)
```
Kolejne argumenty odpowiadają kolejno współrzędnym x, y i z zadanego punktu. 

In [ ]:
trans = SE3(0.5, 0.0, 1.7)
trans

Powstaje w ten sposób macierz jednorodna, której część związana z rotacją jest macierzą jednostkową.
Aby stworzyć odpowiednią macierz rotacji należy skorzystać z polecenia **SE3.OA**.
```py
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
```
Parametry y i z są to:
* y - wektor równoległy do osi y narzędzia
* z - wektor równoległy do osi z narzędzia

In [ ]:
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
rot

Warto zwrócić uwagę, że wektory y i z nie mogą być zerowe lub rónoległe. Nie ma jednak konieczności normalizowania wektorów oraz zapewniania ich prostopadłości. W przypadku pary wektorów nieprostopadłych w macierzy wynikowej zostanie zachowany wektor z a wektor y zostanie do niego dopasowany. Takie działanie wynika z tego, że wektor z określa tzw. kierunek podejścia czyli prościej pozycję osi głównej narzędzia, wektor y odpowiada zaś za obrót narzędzia wokół tej osi.

In [ ]:
y = [1,0,0.5]
z = [1,0,0]
rot = SE3.OA(y, z)
rot

Mnożąc otrzymane w ten sposób macierze translacji i rotacji można w łatwy sposób utworzyć pożądaną macierz jednorodną.

In [ ]:
T = trans * rot
T

Rozwiązanie rozwiązania zagadnienia kinematyki odwrotnej można otrzymać wywołując na obiekcie robota metodę **ikine_LM** i podając jako argument macierz jednorodną.
```py
sol = robot.ikine_LM(T)
```
W zwracanym obiekcie *sol* zawarte są informacje o rozwiązaniu problemu, w tym współrzędne złączowe i informacja o tym czy udało się osiągnąć zadaną pozycję.

In [ ]:
sol = robot.ikine_LM(T)
sol

W przykładzie powyżej widać, że wyliczony wektor współrzędnych złączowych to \[1.458, -0.5544\]. Jednak parametr success=False oznacza, że zadana pozycja nie została osiągnięta.

In [ ]:
print(sol.success)
print(sol.q)

W tym przypadku problem wynika z bardzo prostej struktury manipulatora, która daje bardzo małe możliwości ruchu. W niektórych przypadkach, takich jak ten, zachowanie wszystkich ograniczeń odnośnie pozycji końcowej może być niemożliwe lub niepotrzebne. Narzędzie przewiduje takie sytuacje i daje możliwość określenie w metodzie **ikine_LM** maski odpowiadającej za to, które elementy zadanej pozycji muszą zostać dokładnie osiągnięte. Maska powinna być sześcioelementową tablicą zer i jedynek, w której kolejne elementy oznaczają konieczność dokładnego odwzorowania kolejno: pozycji x, y, i z oraz obrotów wokół osi x, y i z.
```py
mask = np.array([0, 1, 1, 0, 0, 0])   # dokładne odwzorowanie pozycjo w osi y i z
sol = robot.ikine_LM(T, mask=mask)
```
Warto pamiętać, że ilość wartości 1 w masce nie może większa niż ilość stopni swobody manipulatora.

In [ ]:
mask = np.array([0, 1, 1, 0, 0, 0])
sol = robot.ikine_LM(T, mask=mask)
sol

In [ ]:
print(sol.success)
print(sol.q)

Często podanie tylko zadanej pozycji może nie być wystarczające do znalezienia rozwiązania, mimo że pozycja jest osiągalna. Jeżeli jest to możliwe warto również do metody **ikine_LM** podać wartość początkową zmiennych złączowych.
```py
mask = np.array([0, 1, 1, 0, 0, 0])
q0 = np.array([0.0, 1.0])
sol = robot.ikine_LM(T, q0=q0, mask=mask)

```

In [ ]:
mask = np.array([0, 1, 1, 0, 0, 0])
q0 = np.array([0.0, 1.0])
sol = robot.ikine_LM(T, q0=q0, mask=mask)
print(sol.success)
print(sol.q)